In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
df = pd.read_csv('../results/log_bleak.csv')
df

,dir,rated_baud,packet_size,delay,valid,loss_percent,rx_bits,rx_baud
0,BLE >> UART,9600,32,0.000,False,58.672726,5331,6664
1,UART >> BLE,9600,32,0.000,True,0.000000,7650,9562
2,BLE >> UART,9600,32,0.016,False,54.780074,5133,6416
3,UART >> BLE,9600,32,0.016,True,0.000000,7666,9583
4,BLE >> UART,9600,32,0.032,False,18.863071,5917,7396
5,UART >> BLE,9600,32,0.032,True,0.000000,7669,9586
6,BLE >> UART,9600,32,0.064,False,0.025722,3996,4995
7,UART >> BLE,9600,32,0.064,True,0.000000,3985,4982
8,BLE >> UART,19200,32,0.000,False,36.697248,9434,11792
9,UART >> BLE,19200,32,0.000,False,18.391494,12373,15466


In [4]:
ble = df[df.dir.str.startswith('BLE')]
uart = df[df.dir.str.startswith('UART')]

In [5]:
p = uart.pivot(index='rated_baud', columns='delay', values=['loss_percent', 'rx_baud'])

In [7]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for s in p.loss_percent:
    fig.add_scatter(x=p.index, y=p.rx_baud[s], name=f'{s//0.001} ms', secondary_y=True)
    fig.add_bar(x=p.index, y=p.loss_percent[s], name=f'{1/s*32*10 if s > 0 else "∞"} b/s')
fig.update_layout(title='BLE ➟ UART: max. bandwidth', template='plotly_dark', width=900)
fig.update_xaxes(type='category', title='baud rate')
fig.update_yaxes(ticksuffix='%', title='packet loss', range=[100,0], secondary_y=False)
fig.update_yaxes(ticksuffix='b/s', title='bandwidth', range=[0, 20e3], secondary_y=True)
fig.add_annotation()
fig.show()

In [20]:
p.columns

MultiIndex([('loss_percent',   0.0),
            ('loss_percent', 0.016),
            ('loss_percent', 0.032),
            ('loss_percent', 0.064),
            (     'rx_baud',   0.0),
            (     'rx_baud', 0.016),
            (     'rx_baud', 0.032),
            (     'rx_baud', 0.064)],
           names=[None, 'delay'])